# Семинар №7
## Классы (часть 1)
  
  
Сегодня в меню:
  * Создание классов, аттрибуты и методы
  * наследование
  * name mangling
  * iterators, generators (опять)
  * staticmethod, classmethod
  * callable objects

### Создание классов, аттрибуты и методы

In [1]:
class Class:
    pass

instance = Class()
instance.some_str = "instance str"  # динамическое добавление поля объекту

print(instance.some_str)
print(isinstance(instance, Class))  # проверка принадлежности объекта классу

instance str
True


In [2]:
another_instance = Class()
print(another_instance.some_str)

AttributeError: 'Class' object has no attribute 'some_str'

In [3]:
help(Class)

Help on class Class in module __main__:

class Class(builtins.object)
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



__Q:__ Это не очень хорошо, где конструктор (как в С++), где методы, где ссылка на текущий объект?  
__A:__ **Они есть!**

In [4]:
class Class:
    
    def __init__(self, some_str_: str = "I'm instance's field"):  # особый "магический метод" aka конструктор
        self.some_str = some_str_

instance = Class()
print(instance.some_str)
another_instance = Class("I'm another instance's field")
print(another_instance.some_str)
print(instance.some_str)

I'm instance's field
I'm another instance's field
I'm instance's field


In [5]:
class Class:
    
    static_some_str = "I'm static field"
    
    def __init__(self, some_str_: str = "I'm instance's field"):
        self.some_str = some_str_
        
    def print_some_str(self):
        print(f"Class instance field `some_str`='{self.some_str}'")

instance = Class("I'm another instance's field")
instance.print_some_str()

Class instance field `some_str`='I'm another instance's field'


__Q:__ А а что значит self в определении метода?

__A:__ Когда мы вызываем метод как ```<objname>.<methodname>(<arg1>, ..., <argN>)```, нулевым аргументом передается ссылка на ```obj``` (в качестве ```self```)

In [6]:
class MyLittleClass:
    
    def method_without_self(arg):
        print(arg)
        
    def method_with_self(self, arg):
        print(arg)

In [7]:
obj = MyLittleClass()
obj.method_with_self('i am an argument')
obj.method_without_self('i am another argument') # здесь мы на самом деле передаем по два аргумента, self и arg

i am an argument


TypeError: method_without_self() takes 1 positional argument but 2 were given

__Q:__ А как же тогда их вызывать?!

__A:__ Они не привязаны к инстансу (потому что не имеют доступа к его локальным данным), зато привязаны к классу

In [8]:
MyLittleClass.method_without_self('i am another argument')  # а здесь мы передаем только один аргумент

i am another argument


__Q:__ Можно ли "оторвать" метод от инстанса?

__A:__ Ну, попробуем

In [9]:
func = MyLittleClass.method_without_self
func("hello")

hello


In [10]:
func2 = MyLittleClass.method_with_self
func2("hello")  # передаем один аргумент

TypeError: method_with_self() missing 1 required positional argument: 'arg'

In [11]:
func3 = obj.method_with_self
func3("hello")  # передаем два аргумента

hello


In [12]:
func2(MyLittleClass(), "hello")  # ой, нам же ещё нужен объект для self!

hello


__Q:__ А наоборот?

__A:__ Да это же питон. Конечно, можно!

In [13]:
obj.get_color()

AttributeError: 'MyLittleClass' object has no attribute 'get_color'

In [14]:
def get_color_function(self):
    return self.color

MyLittleClass.get_color = get_color_function
obj = MyLittleClass()
obj.get_color()

AttributeError: 'MyLittleClass' object has no attribute 'color'

Ах да, цвета-то у нас нет. Но не беда, это же питон!

In [15]:
obj.color = 'pink'
obj.get_color()

'pink'

__Q:__ А как же узнать, что мы уже определили, а что нет?

__A:__ Легко!

In [16]:
print(dir(obj))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'color', 'get_color', 'method_with_self', 'method_without_self']


In [17]:
# оставим только методы
print([name for name in dir(obj) if callable(getattr(obj, name))])

['__class__', '__delattr__', '__dir__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'get_color', 'method_with_self', 'method_without_self']


In [18]:
class ClassWithNothing:
    pass

nobject = ClassWithNothing()

def print_custom_attrs(obj=None):
    if obj is None:
        # в локальной области видимости!
        attrs = dir()
    else:
        attrs = dir(obj)
    print([name for name in attrs if not name.startswith('__')])
    
print_custom_attrs(nobject)
print_custom_attrs(ClassWithNothing)
print_custom_attrs()
print(dir())

[]
[]
['obj']
['Class', 'ClassWithNothing', 'In', 'MyLittleClass', 'Out', '_', '_15', '__', '___', '__builtin__', '__builtins__', '__doc__', '__loader__', '__name__', '__package__', '__spec__', '_dh', '_i', '_i1', '_i10', '_i11', '_i12', '_i13', '_i14', '_i15', '_i16', '_i17', '_i18', '_i2', '_i3', '_i4', '_i5', '_i6', '_i7', '_i8', '_i9', '_ih', '_ii', '_iii', '_oh', 'another_instance', 'exit', 'func', 'func2', 'func3', 'get_color_function', 'get_ipython', 'instance', 'nobject', 'obj', 'print_custom_attrs', 'quit']


In [19]:
help(dir)

Help on built-in function dir in module builtins:

dir(...)
    dir([object]) -> list of strings
    
    If called without an argument, return the names in the current scope.
    Else, return an alphabetized list of names comprising (some of) the attributes
    of the given object, and of attributes reachable from it.
    If the object supplies a method named __dir__, it will be used; otherwise
    the default dir() logic is used and returns:
      for a module object: the module's attributes.
      for a class object:  its attributes, and recursively the attributes
        of its bases.
      for any other object: its attributes, its class's attributes, and
        recursively the attributes of its class's base classes.



In [20]:
ClassWithNothing.my_attribute = 'my value'
nobject.my_instance_attribute = "my value 2"

print_custom_attrs(nobject)
print_custom_attrs(ClassWithNothing)

['my_attribute', 'my_instance_attribute']
['my_attribute']


### наследование

In [21]:
class Animal:

    some_value = "animal"

    def __init__(self):
        print("i am an animal")
    
    def speak(self):
        raise NotImplementedError('i don\'t know how to speak')

        
class Cat(Animal):
    
    some_value = "cat"
    
    def __init__(self):
        super().__init__()
        print("i am a cat")
    
    def speak(self):
        print('meoooow')

        
class Hedgehog(Animal):
    
    def __init__(self):
        super().__init__()
        print("i am a hedgehog")

        
class Dog(Animal):
    
    some_value = "dog"
    
    def __init__(self):
        super().__init__()
        print("i am a dog")

        
class CatDog(Cat, Dog):  # ромбовидное наследование возможно, но не делайте так, пожалуйста!
    
    def __init__(self):
        super().__init__()
        print("i am a CatDog!")

In [22]:
animal = Animal()
animal.some_value

i am an animal


'animal'

In [23]:
cat = Cat()
cat.some_value  # переопределено

i am an animal
i am a cat


'cat'

In [24]:
hedgehog = Hedgehog()
hedgehog.some_value  # не переопределено

i am an animal
i am a hedgehog


'animal'

In [25]:
dog = Dog()
dog.some_value  # переопределено

i am an animal
i am a dog


'dog'

In [26]:
catdog = CatDog()
catdog.some_value

i am an animal
i am a dog
i am a cat
i am a CatDog!


'cat'

__Q:__ А как определяется порядок?
    
__A:__ Порядок перечисления родителей важен!

In [27]:
class CatDog(Dog, Cat):  # теперь наоборот, найдите два отличия!
    def __init__(self):
        super().__init__()
        print("i am a CatDog!")

catdog = CatDog()
catdog.some_value

i am an animal
i am a cat
i am a dog
i am a CatDog!


'dog'

##### << Пояснительная бригада: начало>>

In [28]:
class A(object):

    def print(self):
        print("Class А")
    pass


class B(A):
    
    def print(self):
        super().print()
        print("Class B")

    pass


class C(A):

    def print(self):
        super().print()
        print("Class C")
    
    pass


class D(B, C):

    pass

In [29]:
d = D()
d.print()

Class А
Class C
Class B


In [30]:
D.mro()

[__main__.D, __main__.B, __main__.C, __main__.A, object]

In [31]:
class E(C, B):

    pass

class F(D, E):

    pass

TypeError: Cannot create a consistent method resolution
order (MRO) for bases B, C

##### << Пояснительная бригада: конец>>

__Q:__ А что с методами?
    
__A:__ Всё то же, что и с атрибутами!

### name mangling

In [32]:
class VeryPrivateDataHolder:
    def __init__(self):
        self._secret = 1
        self.__very_secret = 2

In [33]:
obj = VeryPrivateDataHolder()
print(obj._secret)
print(obj.__very_secret)

1


AttributeError: 'VeryPrivateDataHolder' object has no attribute '__very_secret'

__Q:__ То есть, в питоне всё-таки есть приватность?

__A:__ Ну...

Любой атрибут внутри определения класса ```classname``` вида ```__{attr}``` (```attr``` при этом имеет не более одного ```_``` в конце) подменяется на ```_{classname}__{attr}```. Таким образом, внутри классов можно иметь "скрытые" приватные атрибуты, которые не "видны" экземплярам класса и классам-наследникам.

In [34]:
obj._VeryPrivateDataHolder__very_secret  # а так вообще никогда не делайте, особенно с чужими классами

2

In [35]:
obj._VeryPrivateDataHolder__very_secret = 'new secret'
obj._VeryPrivateDataHolder__very_secret

'new secret'

In [36]:
class DerivedVeryPrivateDataHolder(VeryPrivateDataHolder):
    def __init__(self):
        super().__init__()

obj = DerivedVeryPrivateDataHolder()

In [37]:
print(obj._secret)
print(obj.__very_secret)

1


AttributeError: 'DerivedVeryPrivateDataHolder' object has no attribute '__very_secret'

In [38]:
print(obj._VeryPrivateDataHolder__very_secret)
print(obj._DerivedVeryPrivateDataHolder__very_secret)

2


AttributeError: 'DerivedVeryPrivateDataHolder' object has no attribute '_DerivedVeryPrivateDataHolder__very_secret'

### iterators, generators

In [39]:
class ForwardRange:
    def __init__(self, n_max):
        self.i = 0
        self.n_max = n_max

    def __iter__(self):
        return self

    def __next__(self):
        if self.i < self.n_max:
            i = self.i
            self.i += 1
            return i
        else:
            # специальное исключение, которое означает "элементы кончились!"
            # впрочем, может никогда и не бросаться
            raise StopIteration()

In [40]:
iterator_obj = ForwardRange(3)
print(iterator_obj.__next__())
print(iterator_obj.__next__())
print(iterator_obj.__next__())
print(iterator_obj.__next__())
print(iterator_obj.__next__())

0
1
2


StopIteration: 

__Q:__ И что, чтобы им пользоваться, надо ловить исключения?

__A:__ Конечно, нет! Это non-pythonic way

In [41]:
iterator_obj = ForwardRange(3)
print(type(iterator_obj))
for x in iterator_obj:
    print(x)

<class '__main__.ForwardRange'>
0
1
2


In [42]:
for x in iterator_obj:
    print(x)

__Q:__ Повторно использовать нельзя?!

__A:__ Объект итератора, как можно понять из кода, хранит своё состояние. Он уже выдал нам всё, что должен был

In [43]:
def forward_range(n_max):
    i = 0
    while i < n_max:
        yield i
        i += 1

In [44]:
generator_obj = forward_range(3)
print(type(generator_obj))
# мы не определяли магических функций итератора, но...
print(generator_obj.__iter__)
print(generator_obj.__iter__())
print(generator_obj.__next__)

<class 'generator'>
<method-wrapper '__iter__' of generator object at 0x7f245853a040>
<generator object forward_range at 0x7f245853a040>
<method-wrapper '__next__' of generator object at 0x7f245853a040>


In [45]:
for x in generator_obj:
    print(x)

0
1
2


In [46]:
for x in generator_obj:
    print(x)

__Q:__ А чем отличается практическое использование?

__A:__ Как правило, почти ничем, если нет избыточной логики итерации

### staticmethod, classmethod

In [47]:
class MyClass:
    
    clsval = 0
    
    def __init__(self,val):
        self.objval = val

    def set_val(self,val):
        type(self).clsval = val  # атрибут класса 
        self.objval = val        # атрибут объекта 
    
    @staticmethod  # можно вызывать и как obj.static_set_val(val) и как MyClass.static_set_val(val)!
    def static_set_val(val):
        MyClass.clsval = val
        
    @classmethod  # передаёт класс первым аргументом
    def class_set_val(cls, val):
        cls.clsval = val

In [48]:
obj = MyClass(5)
print('clsval', obj.clsval, 'objval', obj.objval)

obj.set_val(9)
print('clsval', obj.clsval, 'objval', obj.objval)

obj.static_set_val(4)
print('clsval', obj.clsval, 'objval', obj.objval)

MyClass.static_set_val(3)
print('clsval', obj.clsval, 'objval', obj.objval)

MyClass.class_set_val(7)
print('clsval', obj.clsval, 'objval', obj.objval)

clsval 0 objval 5
clsval 9 objval 9
clsval 4 objval 9
clsval 3 objval 9
clsval 7 objval 9


### callable objects

In [49]:
class Adder:

    def __init__(self, x):
        self.x = x

    def __call__(self, y):
        return self.x + y
    
adder = Adder(10)

adder(14)

24